# Glacier grids from RGI:

Creates monthly grid files for the MBM to make PMB predictions over the whole glacier grid. The files come from the RGI grid with OGGM topography. Computing takes a long time because of the conversion to monthly format.
## Setting up:

In [ ]:
# --- System & utilities ---
import os
import sys
import re
import csv
import ast
import math
import traceback
import itertools
import random
import pickle
import logging
import warnings
from datetime import datetime
from functools import partial
from collections import Counter, defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed

# Add repo root for MBM imports
sys.path.append(os.path.join(os.getcwd(), "../../"))

# --- Data science stack ---
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from cmcrameri import cm

# --- Machine learning / DL ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from skorch.helper import SliceDataset
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint

# --- Cartography / plotting ---
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# --- Custom MBM modules ---
import massbalancemachine as mbm
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *

# --- Warnings & autoreload (notebook) ---
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

# --- Configuration ---
cfg = mbm.SwitzerlandConfig()
seed_all(cfg.seed)
print("Using seed:", cfg.seed)

# --- CUDA / device ---
if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# seed_all(cfg.seed)
# free_up_cuda()  # in case no memory

# # Plot styles:
# path_style_sheet = 'scripts/example.mplstyle'
# plt.style.use(path_style_sheet)

# # Climate columns
# vois_climate = [
#     't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
# ]
# # Topographical columns
# vois_topographical = [
#     "aspect",
#     "slope",
#     "hugonnet_dhdt",
#     "consensus_ice_thickness",
#     "millan_v",
#     "topo",
# ]

glacier_outline_rgi = gpd.read_file(cfg.dataPath + path_rgi_outlines)

In [ ]:
gdirs, rgidf = initialize_oggm_glacier_directories(
    cfg,
    rgi_region="11",
    rgi_version="6",
    # base_url=
    # "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/",
    base_url=
    "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/2025.6/elev_bands_w_data/",
    log_level='WARNING',
    task_list=None,
)

# Save OGGM xr for all needed glaciers in RGI region 11.6:
df_missing = export_oggm_grids(cfg, gdirs)

path_rgi = cfg.dataPath + 'GLAMOS/RGI/nsidc0770_11.rgi60.CentralEurope/11_rgi60_CentralEurope.shp'

# load RGI shapefile
gdf = gpd.read_file(path_rgi)
# reproject to a local equal-area projection (example: EPSG:3035 for Europe)
gdf_proj = gdf.to_crs(3035)
gdf_proj.rename(columns={"RGIId": "rgi_id"}, inplace=True)
# gdf_proj.set_index('rgi_id', inplace=True)
gdf_proj["area_m2"] = gdf_proj.geometry.area
gdf_proj["area_km2"] = gdf_proj["area_m2"] / 1e6

df_missing = df_missing.merge(gdf_proj[['area_km2', 'rgi_id']], on="rgi_id")

# total glacier area
total_area = gdf_proj["area_km2"].sum()

# explode the list of missing vars into rows (one var per row)
df_exploded = df_missing.explode("missing_vars")

# 1) COUNT: number of glaciers missing each variable
counts_missing_per_var = (
    df_exploded.groupby("missing_vars")["rgi_id"].nunique().sort_values(
        ascending=False))

# 2) TOTAL % AREA with ANY missing var
total_missing_area_km2 = df_missing["area_km2"].sum()
total_missing_area_pct = (total_missing_area_km2 / total_area) * 100

print(f"Total glacier area with ANY missing variable: "
      f"{total_missing_area_km2:,.2f} km² "
      f"({total_missing_area_pct:.2f}%)")

# Optional: also show % area per variable (kept from your earlier logic)
area_missing_per_var = (
    df_exploded.groupby("missing_vars")["area_km2"].sum().sort_values(
        ascending=False))
perc_missing_per_var = (area_missing_per_var / total_area) * 100

print("\n% of total glacier area missing per variable:")
for var, pct in perc_missing_per_var.items():
    print(f"  - {var}: {pct:.2f}%")

# ---- barplot: number of glaciers missing each variable ----
plt.figure(figsize=(7, 4))
plt.bar(counts_missing_per_var.index, counts_missing_per_var.values)
plt.xlabel("Missing variable")
plt.ylabel("Number of glaciers")
plt.title("Count of glaciers missing each variable")
plt.tight_layout()
plt.show()

In [ ]:
# RGI Ids:
# Read glacier ids:
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)
rgi_df.loc['rhone']

### Location of all glaciers:

In [ ]:
path_rgi_alps = os.path.join(cfg.dataPath,
                             'GLAMOS/topo/gridded_topo_inputs/RGI_v6_11_svf/')
year = 2022
rgi_ids = os.listdir(path_rgi_alps)
pos_gl, rgis = [], []
for rgi_gl in tqdm(rgi_ids):
    # if path exists:
    if os.path.exists(
            os.path.join(path_rgi_alps, rgi_gl,
                         f"{rgi_gl}_grid_{year}.parquet")):
        df = pd.read_parquet(
            os.path.join(path_rgi_alps, rgi_gl,
                         f"{rgi_gl}_grid_{year}.parquet"))
        pos_gl.append((df.POINT_LAT.mean(), df.POINT_LON.mean()))
        rgis.append(rgi_gl)
    else:
        continue
df_pos_all = pd.DataFrame(pos_gl, columns=['lat', 'lon'])
df_pos_all['rgi_id'] = rgis

print('Number of glaciers in RGI region 11.6:', len(df_pos_all))

# ---- 2. Create figure and base map ----
fig = plt.figure(figsize=(18, 10))

latN, latS = 48, 44
lonW, lonE = 4, 14
projPC = ccrs.PlateCarree()
ax2 = plt.axes(projection=projPC)
ax2.set_extent([lonW, lonE, latS, latN], crs=ccrs.Geodetic())

ax2.add_feature(cfeature.COASTLINE)
ax2.add_feature(cfeature.LAKES)
ax2.add_feature(cfeature.RIVERS)
ax2.add_feature(cfeature.BORDERS, linestyle='-', linewidth=1)

g = sns.scatterplot(
    data=df_pos_all,
    x='lon',
    y='lat',
    alpha=0.6,
    transform=projPC,
    ax=ax2,
    zorder=10,
    legend=True  # custom legend added below
)

glacier_outline_rgi.plot(ax=ax2, transform=projPC, color='black')

# ---- 4. Gridlines ----
gl = ax2.gridlines(draw_labels=True,
                   linewidth=1,
                   color='gray',
                   alpha=0.5,
                   linestyle='--')
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 16, 'color': 'black'}
gl.ylabel_style = {'size': 16, 'color': 'black'}
gl.top_labels = gl.right_labels = False

In [ ]:
df = pd.read_parquet(
    os.path.join(path_rgi_alps, rgi_gl, f"{rgi_gl}_grid_{year}.parquet"))
df.columns

## Train LSTM model:

In [ ]:
data_glamos = getStakesData(cfg)

months_head_pad, months_tail_pad = mbm.data_processing.utils._compute_head_tail_pads_from_df(
    data_glamos)

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}

# Climate columns
vois_climate = [
    't2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'str', 'u10', 'v10'
]
# Topographical columns
vois_topographical = [
    "aspect", "slope", "hugonnet_dhdt", "consensus_ice_thickness", "millan_v",
    "topo", "SVF"
]

RUN = False
data_monthly = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_LSTM_CA.csv')

# Create DataLoader
dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(data_monthly.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

if missing_glaciers:
    print(
        f"Warning: The following test glaciers are not in the dataset: {missing_glaciers}"
    )

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = data_monthly[data_monthly.GLACIER.isin(TEST_GLACIERS)]
print('Size of monthly test data:', len(data_test))

data_train = data_monthly[data_monthly.GLACIER.isin(train_glaciers)]
print('Size of monthly train data:', len(data_train))

if len(data_train) == 0:
    print("Warning: No training data available!")
else:
    test_perc = (len(data_test) / len(data_train)) * 100
    print('Percentage of test size: {:.2f}%'.format(test_perc))

splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)

print('Test glaciers: ({}) {}'.format(len(test_set['splits_vals']),
                                      test_set['splits_vals']))
test_perc = (len(test_set['df_X']) / len(train_set['df_X'])) * 100
print('Percentage of test size: {:.2f}%'.format(test_perc))
print('Size of test set:', len(test_set['df_X']))
print('Train glaciers: ({}) {}'.format(len(train_set['splits_vals']),
                                       train_set['splits_vals']))
print('Size of train set:', len(train_set['df_X']))

# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']

data_test = test_set['df_X']
data_test['y'] = test_set['y']

### Simple model:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'ELEVATION_DIFFERENCE',
]
STATIC_COLS = ['aspect', 'slope', 'SVF']

feature_columns = MONTHLY_COLS + STATIC_COLS
seed_all(cfg.seed)

df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# --- build train dataset from dataframe ---
ds_train_simple = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

ds_test_simple = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

train_idx_simple, val_idx_simple = mbm.data_processing.MBSequenceDataset.split_indices(
    len(ds_train_simple), val_ratio=0.2, seed=cfg.seed)

custom_params = {
    'Fm': len(MONTHLY_COLS),
    'Fs': len(STATIC_COLS),
    'hidden_size': 128,
    'num_layers': 1,
    'bidirectional': False,
    'dropout': 0.0,
    'static_layers': 0,
    'static_hidden': None,
    'static_dropout': None,
    'lr': 0.001,
    'weight_decay': 0.0,
    'loss_name': 'neutral',
    'loss_spec': None
}

custom_params['two_heads'] = True
custom_params['head_dropout'] = 0.0

params_simple_model = custom_params.copy()

# --- build model, resolve loss, train, reload best ---
current_date = datetime.now().strftime("%Y-%m-%d")
model_filename = f"models/lstm_model_{current_date}_CA_simple.pt"

# --- loaders (fit scalers on TRAIN, apply to whole ds_train_simple) ---
ds_train_simple_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_train_simple)

ds_test_simple_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_test_simple)

train_dl, val_dl = ds_train_simple_copy.make_loaders(
    train_idx=train_idx_simple,
    val_idx=val_idx_simple,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# --- test loader (copies TRAIN scalers into ds_test_simple and transforms it) ---
test_dl_simple = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test_simple_copy, ds_train_simple_copy, batch_size=128, seed=cfg.seed)

# --- build model, resolve loss, train, reload best ---
model = mbm.models.LSTM_MB.build_model_from_params(cfg, custom_params, device)
loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

TRAIN = False
if TRAIN:
    if os.path.exists(model_filename): os.remove(model_filename)

    history, best_val, best_state = model.train_loop(
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        epochs=150,
        lr=custom_params['lr'],
        weight_decay=custom_params['weight_decay'],
        clip_val=1,
        # scheduler
        sched_factor=0.5,
        sched_patience=6,
        sched_threshold=0.01,
        sched_threshold_mode="rel",
        sched_cooldown=1,
        sched_min_lr=1e-6,
        # early stopping
        es_patience=15,
        es_min_delta=1e-4,
        # logging
        log_every=5,
        verbose=True,
        # checkpoint
        save_best_path=model_filename,
        loss_fn=loss_fn,
    )
    plot_history_lstm(history)

model_filename = f"models/lstm_model_2025-10-07_CA_simple.pt"
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state)

# Evaluate on test
test_metrics, test_df_preds = model.evaluate_with_preds(
    device, test_dl_simple, ds_test_simple_copy)
test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
    'RMSE_winter']

print('Test RMSE annual: {:.3f} | winter: {:.3f}'.format(
    test_rmse_a, test_rmse_w))

scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

fig = plot_predictions_summary(grouped_ids=test_df_preds,
                               scores_annual=scores_annual,
                               scores_winter=scores_winter,
                               ax_xlim=(-8, 6),
                               ax_ylim=(-8, 6))

### Partial models:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'ELEVATION_DIFFERENCE',
]
vars = ['hugonnet_dhdt', 'consensus_ice_thickness', 'millan_v']

all_combos = [('hugonnet_dhdt', ), ('consensus_ice_thickness', ),
              ('millan_v', ), ('hugonnet_dhdt', 'consensus_ice_thickness'),
              ('hugonnet_dhdt', 'millan_v'),
              ('consensus_ice_thickness', 'millan_v')]

TRAIN = False
if TRAIN:
    for combo in all_combos:
        print(combo)

        STATIC_COLS = ['aspect', 'slope', 'SVF', *combo]
        feature_columns = MONTHLY_COLS + STATIC_COLS
        seed_all(cfg.seed)

        # prepare train/test data
        df_train = data_train.copy()
        df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()
        df_test = data_test.copy()
        df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

        # datasets
        ds_train_full = mbm.data_processing.MBSequenceDataset.from_dataframe(
            df_train,
            MONTHLY_COLS,
            STATIC_COLS,
            months_tail_pad=months_tail_pad,
            months_head_pad=months_head_pad,
            expect_target=True)
        ds_test_full = mbm.data_processing.MBSequenceDataset.from_dataframe(
            df_test,
            MONTHLY_COLS,
            STATIC_COLS,
            months_tail_pad=months_tail_pad,
            months_head_pad=months_head_pad,
            expect_target=True)

        # split train/val
        train_idx_full, val_idx_full = mbm.data_processing.MBSequenceDataset.split_indices(
            len(ds_train_full), val_ratio=0.2, seed=cfg.seed)

        # params
        custom_params = {
            'Fm': len(MONTHLY_COLS),
            'Fs': len(STATIC_COLS),
            'hidden_size': 128,
            'num_layers': 1,
            'bidirectional': False,
            'dropout': 0.0,
            'static_layers': 0,
            'static_hidden': None,
            'static_dropout': None,
            'lr': 0.001,
            'weight_decay': 0.0,
            'loss_name': 'neutral',
            'loss_spec': None,
            'two_heads': True,
            'head_dropout': 0.0
        }
        params_full_model = custom_params.copy()

        # --- model filename ---
        current_date = datetime.now().strftime("%Y-%m-%d")
        combo_str = "_".join(combo)
        model_filename = f"models/lstm_model_{current_date}_CA_{combo_str}.pt"

        # loaders
        ds_train_full_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
            ds_train_full)
        ds_test_full_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
            ds_test_full)

        train_dl, val_dl = ds_train_full_copy.make_loaders(
            train_idx=train_idx_full,
            val_idx=val_idx_full,
            batch_size_train=64,
            batch_size_val=128,
            seed=cfg.seed,
            fit_and_transform=True,
            shuffle_train=True,
            use_weighted_sampler=True)

        test_dl_full = mbm.data_processing.MBSequenceDataset.make_test_loader(
            ds_test_full_copy,
            ds_train_full_copy,
            batch_size=128,
            seed=cfg.seed)

        # model
        model = mbm.models.LSTM_MB.build_model_from_params(
            cfg, custom_params, device)
        loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

        if os.path.exists(model_filename):
            os.remove(model_filename)

        history, best_val, best_state = model.train_loop(
            device=device,
            train_dl=train_dl,
            val_dl=val_dl,
            epochs=150,
            lr=custom_params['lr'],
            weight_decay=custom_params['weight_decay'],
            clip_val=1,
            sched_factor=0.5,
            sched_patience=6,
            sched_threshold=0.01,
            sched_threshold_mode="rel",
            sched_cooldown=1,
            sched_min_lr=1e-6,
            es_patience=15,
            es_min_delta=1e-4,
            log_every=5,
            verbose=True,
            save_best_path=model_filename,
            loss_fn=loss_fn)

#### Full model (with OGGM variables):

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'ELEVATION_DIFFERENCE',
]

STATIC_COLS = [
    'aspect', 'slope', 'SVF', 'hugonnet_dhdt', 'consensus_ice_thickness',
    'millan_v'
]

feature_columns = MONTHLY_COLS + STATIC_COLS
seed_all(cfg.seed)

df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# --- build train dataset from dataframe ---
ds_train_full = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

ds_test_full = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

train_idx_full, val_idx_full = mbm.data_processing.MBSequenceDataset.split_indices(
    len(ds_train_full), val_ratio=0.2, seed=cfg.seed)

custom_params = {
    'Fm': len(MONTHLY_COLS),
    'Fs': len(STATIC_COLS),
    'hidden_size': 128,
    'num_layers': 1,
    'bidirectional': False,
    'dropout': 0.0,
    'static_layers': 0,
    'static_hidden': None,
    'static_dropout': None,
    'lr': 0.001,
    'weight_decay': 0.0,
    'loss_name': 'neutral',
    'loss_spec': None
}

custom_params['two_heads'] = True
custom_params['head_dropout'] = 0.0

params_full_model = custom_params.copy()

# --- build model, resolve loss, train, reload best ---
current_date = datetime.now().strftime("%Y-%m-%d")
model_filename = f"models/lstm_model_{current_date}_CA_full.pt"

# --- loaders (fit scalers on TRAIN, apply to whole ds_train_full) ---
ds_train_full_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_train_full)

ds_test_full_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_test_full)

train_dl, val_dl = ds_train_full_copy.make_loaders(
    train_idx=train_idx_full,
    val_idx=val_idx_full,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# --- test loader (copies TRAIN scalers into ds_test_full and transforms it) ---
test_dl_full = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test_full_copy, ds_train_full_copy, batch_size=128, seed=cfg.seed)

# --- build model, resolve loss, train, reload best ---
model = mbm.models.LSTM_MB.build_model_from_params(cfg, custom_params, device)
loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

TRAIN = False
if TRAIN:
    if os.path.exists(model_filename): os.remove(model_filename)

    history, best_val, best_state = model.train_loop(
        device=device,
        train_dl=train_dl,
        val_dl=val_dl,
        epochs=150,
        lr=custom_params['lr'],
        weight_decay=custom_params['weight_decay'],
        clip_val=1,
        # scheduler
        sched_factor=0.5,
        sched_patience=6,
        sched_threshold=0.01,
        sched_threshold_mode="rel",
        sched_cooldown=1,
        sched_min_lr=1e-6,
        # early stopping
        es_patience=15,
        es_min_delta=1e-4,
        # logging
        log_every=5,
        verbose=True,
        # checkpoint
        save_best_path=model_filename,
        loss_fn=loss_fn,
    )
    plot_history_lstm(history)

model_filename = f"models/lstm_model_2025-10-07_CA_full.pt"

state = torch.load(model_filename, map_location=device)
model.load_state_dict(state)

# Evaluate on test
test_metrics, test_df_preds = model.evaluate_with_preds(
    device, test_dl_full, ds_test_full_copy)
test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
    'RMSE_winter']

print('Test RMSE annual: {:.3f} | winter: {:.3f}'.format(
    test_rmse_a, test_rmse_w))

scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

fig = plot_predictions_summary(grouped_ids=test_df_preds,
                               scores_annual=scores_annual,
                               scores_winter=scores_winter,
                               ax_xlim=(-8, 6),
                               ax_ylim=(-8, 6))

## Extrapolate to all glaciers:

In [ ]:
import copy

# fixed order to avoid accidental shuffles
VARS_ORDER = ['hugonnet_dhdt', 'consensus_ice_thickness', 'millan_v']

MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'ELEVATION_DIFFERENCE',
]

all_combos = [
    ('hugonnet_dhdt', ),
    ('consensus_ice_thickness', ),
    ('millan_v', ),
    ('hugonnet_dhdt', 'consensus_ice_thickness'),
    ('hugonnet_dhdt', 'millan_v'),
    ('consensus_ice_thickness', 'millan_v'),
]


def combo_key(tup):
    ordered = [v for v in VARS_ORDER if v in tup]
    return "__".join(ordered)  # e.g. "hugonnet_dhdt__millan_v"


def make_params(Fm, STATIC_COLS):
    # Your template, with Fs derived from STATIC_COLS length
    return {
        'Fm': Fm,  # number of monthly features
        'Fs': len(STATIC_COLS),  # number of static features
        'hidden_size': 128,
        'num_layers': 1,
        'bidirectional': False,
        'dropout': 0.0,
        'static_layers': 0,
        'static_hidden': None,
        'static_dropout': None,
        'lr': 0.001,
        'weight_decay': 0.0,
        'loss_name': 'neutral',
        'loss_spec': None,
        'two_heads': True,
        'head_dropout': 0.0
    }


Fm = len(MONTHLY_COLS)  # safer than hardcoding 8

# Build params for each requested combo
params_by_key = {}
for combo in all_combos:
    STATIC_COLS = ['aspect', 'slope', 'SVF', *combo]
    key = combo_key(combo)  # e.g. 'hugonnet_dhdt__millan_v'
    params_by_key[key] = make_params(Fm, STATIC_COLS)

# Unpack to the variable names you mentioned
params_hugonnet_only = copy.deepcopy(params_by_key['hugonnet_dhdt'])
params_consensus_only = copy.deepcopy(params_by_key['consensus_ice_thickness'])
params_millan_only = copy.deepcopy(params_by_key['millan_v'])
params_hugonnet_consensus = copy.deepcopy(
    params_by_key['hugonnet_dhdt__consensus_ice_thickness'])
params_hugonnet_millan = copy.deepcopy(
    params_by_key['hugonnet_dhdt__millan_v'])
params_consensus_millan = copy.deepcopy(
    params_by_key['consensus_ice_thickness__millan_v'])

# (optional) One dict for your loader mapping
PARAMS_BY_COMBO = {
    "hugonnet_dhdt": params_hugonnet_only,
    "consensus_ice_thickness": params_consensus_only,
    "millan_v": params_millan_only,
    "hugonnet_dhdt__consensus_ice_thickness": params_hugonnet_consensus,
    "hugonnet_dhdt__millan_v": params_hugonnet_millan,
    "consensus_ice_thickness__millan_v": params_consensus_millan,
}

In [ ]:
import copy

# ---- config / constants ----
VARS_ORDER = ['hugonnet_dhdt', 'consensus_ice_thickness',
              'millan_v']  # fixed order
# Your existing MONTHLY_COLS
# MONTHLY_COLS = ['t2m','tp','slhf','sshf','ssrd','fal','str','ELEVATION_DIFFERENCE']


def combo_key_from_tuple(tup):
    if len(tup) == 0:
        return "simple"
    ordered = [v for v in VARS_ORDER if v in tup]
    if ordered == VARS_ORDER:
        return "full"
    return "__".join(ordered)


# The partial combos you trained
partial_combos = [
    ('hugonnet_dhdt', ),
    ('consensus_ice_thickness', ),
    ('millan_v', ),
    ('hugonnet_dhdt', 'consensus_ice_thickness'),
    ('hugonnet_dhdt', 'millan_v'),
    ('consensus_ice_thickness', 'millan_v'),
]

# Include simple and full for completeness
ALL_COMBOS = [()] + partial_combos + [tuple(VARS_ORDER)]

# ---- normalize input dataframes once ----
df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()  # optional, if you also want test ds per combo
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# ---- builders ----
DS_TRAIN_BY_COMBO = {}
SPLIT_IDXS_BY_COMBO = {}  # (train_idx, val_idx)
STATIC_COLS_BY_COMBO = {}

seed_all(cfg.seed)

for combo in ALL_COMBOS:
    key = combo_key_from_tuple(combo)

    # Build static columns in stable order
    if key == "simple":
        STATIC_COLS = ['aspect', 'slope', 'SVF']
    elif key == "full":
        STATIC_COLS = ['aspect', 'slope', 'SVF', *VARS_ORDER]
    else:
        # enforce VARS_ORDER inside the combo
        ordered = [v for v in VARS_ORDER if v in combo]
        STATIC_COLS = ['aspect', 'slope', 'SVF', *ordered]

    STATIC_COLS_BY_COMBO[key] = STATIC_COLS

    # --- build train dataset from dataframe ---
    ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
        df_train,
        MONTHLY_COLS,
        STATIC_COLS,
        months_tail_pad=months_tail_pad,
        months_head_pad=months_head_pad,
        expect_target=True)
    DS_TRAIN_BY_COMBO[key] = ds_train

    # keep per-combo split (use your preferred val_ratio/seed)
    train_idx, val_idx = mbm.data_processing.MBSequenceDataset.split_indices(
        len(ds_train), val_ratio=0.2, seed=cfg.seed)
    SPLIT_IDXS_BY_COMBO[key] = (train_idx, val_idx)

# ---- (optional) create named variables for convenience, matching your earlier mapping ----
ds_train_simple = DS_TRAIN_BY_COMBO['simple']
ds_train_full = DS_TRAIN_BY_COMBO['full']

ds_train_hugonnet_only = DS_TRAIN_BY_COMBO['hugonnet_dhdt']
ds_train_consensus_only = DS_TRAIN_BY_COMBO['consensus_ice_thickness']
ds_train_millan_only = DS_TRAIN_BY_COMBO['millan_v']
ds_train_hugonnet_consensus = DS_TRAIN_BY_COMBO[
    'hugonnet_dhdt__consensus_ice_thickness']
ds_train_hugonnet_millan = DS_TRAIN_BY_COMBO['hugonnet_dhdt__millan_v']
ds_train_consensus_millan = DS_TRAIN_BY_COMBO[
    'consensus_ice_thickness__millan_v']

# and their splits if you want named access
train_idx_simple, val_idx_simple = SPLIT_IDXS_BY_COMBO['simple']
train_idx_full, val_idx_full = SPLIT_IDXS_BY_COMBO['full']

In [ ]:
# ---------- setup (top of file) ----------
VARS_ORDER = ['hugonnet_dhdt', 'consensus_ice_thickness',
              'millan_v']  # keep a fixed order


def combo_key_from_tuple(tup):
    """('hugonnet_dhdt','millan_v') -> 'hugonnet_dhdt__millan_v' ; () -> 'simple' ; all 3 -> 'full'"""
    if not tup:
        return "simple"
    ordered = [v for v in VARS_ORDER if v in tup]
    if ordered == VARS_ORDER:
        return "full"
    return "__".join(ordered)


# List *exactly* the combos you trained (keys must match your saved filenames/params)
TRAINED_COMBOS = [
    (),  # simple (no OGGM)
    ('hugonnet_dhdt', ),
    ('consensus_ice_thickness', ),
    ('millan_v', ),
    ('hugonnet_dhdt', 'consensus_ice_thickness'),
    ('hugonnet_dhdt', 'millan_v'),
    ('consensus_ice_thickness', 'millan_v'),
    tuple(VARS_ORDER),  # full (all OGGM)
]

# Build mapping keys
COMBO_KEYS = [combo_key_from_tuple(c) for c in TRAINED_COMBOS]

# ---- checkpoints for each combo (update paths to match your training) ----
# Example filename convention used when training: "lstm_model_YYYY-MM-DD_CA_<combo>.pt"
# where <combo> joined by underscores; simple/full named accordingly.
MODEL_PATHS = {
    "simple": "models/lstm_model_2025-10-07_CA_simple.pt",
    "hugonnet_dhdt": "models/lstm_model_2025-10-07_CA_hugonnet_dhdt.pt",
    "consensus_ice_thickness":
    "models/lstm_model_2025-10-07_CA_consensus_ice_thickness.pt",
    "millan_v": "models/lstm_model_2025-10-07_CA_millan_v.pt",
    "hugonnet_dhdt__consensus_ice_thickness":
    "models/lstm_model_2025-10-07_CA_hugonnet_dhdt_consensus_ice_thickness.pt",
    "hugonnet_dhdt__millan_v":
    "models/lstm_model_2025-10-07_CA_hugonnet_dhdt_millan_v.pt",
    "consensus_ice_thickness__millan_v":
    "models/lstm_model_2025-10-07_CA_consensus_ice_thickness_millan_v.pt",
    "full": "models/lstm_model_2025-10-07_CA_full.pt",
}

# ---- params per combo (you already have simple/full; add the partials) ----
# Each dict must match what you used to train that checkpoint (Fs must reflect STATIC_COLS length!)
PARAMS_BY_COMBO = {
    "simple": params_simple_model,  # you already have this
    "full": params_full_model,  # you already have this
    # add your partials (example: copy from your training configs)
    "hugonnet_dhdt": params_hugonnet_only,
    "consensus_ice_thickness": params_consensus_only,
    "millan_v": params_millan_only,
    "hugonnet_dhdt__consensus_ice_thickness": params_hugonnet_consensus,
    "hugonnet_dhdt__millan_v": params_hugonnet_millan,
    "consensus_ice_thickness__millan_v": params_consensus_millan,
}

# ---- training datasets used only to provide the *matching scalers* for each combo ----
# Build them once (or reuse objects you already created after training).
# They must be produced with the *same* MONTHLY_COLS and STATIC_COLS used to train that combo.
TRAIN_DS_BY_COMBO = {
    "simple": ds_train_simple,  # you already had this for simple
    "full": ds_train_full,  # you already had this for full
    "hugonnet_dhdt": ds_train_hugonnet_only,
    "consensus_ice_thickness": ds_train_consensus_only,
    "millan_v": ds_train_millan_only,
    "hugonnet_dhdt__consensus_ice_thickness": ds_train_hugonnet_consensus,
    "hugonnet_dhdt__millan_v": ds_train_hugonnet_millan,
    "consensus_ice_thickness__millan_v": ds_train_consensus_millan,
}

# Cache for loaded models
_MODEL_CACHE = {}


def get_model_by_combo(combo_key: str, cfg, device):
    """Generalized model loader with cache."""
    if combo_key in _MODEL_CACHE:
        return _MODEL_CACHE[combo_key]

    if combo_key not in MODEL_PATHS:
        raise ValueError(f"No model path for combo '{combo_key}'")
    if combo_key not in PARAMS_BY_COMBO:
        raise ValueError(f"No params for combo '{combo_key}'")

    ckpt_path = MODEL_PATHS[combo_key]
    if not os.path.exists(ckpt_path):
        raise FileNotFoundError(
            f"Checkpoint not found for '{combo_key}': {ckpt_path}")

    params = PARAMS_BY_COMBO[combo_key]
    model = mbm.models.LSTM_MB.build_model_from_params(cfg,
                                                       params,
                                                       device,
                                                       verbose=False)
    state = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(state)
    model.eval()
    _MODEL_CACHE[combo_key] = model
    return model


def detect_available_combo_key(df: pd.DataFrame) -> str:
    """
    Return the most specific trained combo that matches columns present in df.
    Priority: full > any 2-var combo > any 1-var combo > simple.
    """
    present = [c for c in VARS_ORDER if c in df.columns]
    present_set = set(present)

    # try exact matches by decreasing size
    candidates = sorted(TRAINED_COMBOS, key=lambda t: len(t), reverse=True)
    for tup in candidates:
        if set(tup).issubset(present_set):
            return combo_key_from_tuple(tup)

    return "simple"

In [ ]:
# simple
STATIC_COLS_simple = ['aspect', 'slope', 'SVF']
params_simple_model = make_params(Fm, STATIC_COLS_simple)
PARAMS_BY_COMBO['simple'] = params_simple_model

# full
STATIC_COLS_full = ['aspect', 'slope', 'SVF', *VARS_ORDER]
params_full_model = make_params(Fm, STATIC_COLS_full)
PARAMS_BY_COMBO['full'] = params_full_model

In [ ]:
# Required by the dataset builder regardless of your feature list
REQUIRED = ['GLACIER', 'YEAR', 'ID', 'PERIOD', 'MONTHS']
# Define paths
# path_save_glw = os.path.join(
#     cfg.dataPath, 'GLAMOS/distributed_MB_grids/MBM/central_europe/')
rgi_id_list = os.listdir(path_rgi_alps)

# Safe rename helper
def safe_rename(df, mapping):
    present = {k: v for k, v in mapping.items() if k in df.columns}
    return df.rename(columns=present) if present else df

# Robust year regex: RGI60-11.01238_grid_2003.parquet
YEAR_RE = re.compile(r"_grid_(\d{4})\.parquet$")

# --- main run loop (replaces your has_oggm/simple branch) ---
RUN = True
if RUN:
    # emptyfolder(path_save_glw)

    output_file = os.path.join(f"logs/glacier_mean_MB_{current_date}.csv")
    with open(output_file, 'w') as f:
        f.write("Index,RGIId,Year,Mean_MB\n")

    index_counter = 0

    for rgi_gl in tqdm(rgi_id_list):
        seed_all(cfg.seed)
        glacier_path = os.path.join(path_rgi_alps, rgi_gl)
        if not os.path.exists(glacier_path):
            print(f"Folder not found for {rgi_gl}, skipping...")
            continue

        # collect available years
        years = []
        for fname in os.listdir(glacier_path):
            if not (fname.endswith(".parquet") and rgi_gl in fname):
                continue
            m = YEAR_RE.search(fname)
            if m:
                years.append(int(m.group(1)))
        years = sorted(set(years))
        if not years:
            print(f"No parquet years for {rgi_gl}, skipping...")
            continue

        # clear model cache per glacier (optional)
        _MODEL_CACHE.clear()

        for year in years:
            file_name = f"{rgi_gl}_grid_{year}.parquet"
            file_path = os.path.join(glacier_path, file_name)

            try:
                df_grid_monthly = pd.read_parquet(file_path)
                df_grid_monthly.rename(columns={'svf':'SVF'}, inplace=True)
            except Exception as e:
                print(f"[{rgi_gl} {year}] read error: {e}")
                continue

            df_grid_monthly.drop_duplicates(inplace=True)

            # harmonize column names if needed
            # df_grid_monthly = safe_rename(df_grid_monthly, {
            #     'aspect': 'aspect_sgi',
            #     'slope': 'slope_sgi',
            # })

            # --- choose the best available trained combo for this file ---
            combo_key = detect_available_combo_key(
                df_grid_monthly
            )  # e.g. 'simple', 'hugonnet_dhdt', 'hugonnet_dhdt__millan_v', 'full'

            # build STATIC_COLS in stable order matching training schema
            if combo_key == "simple":
                oggm_cols = []
            elif combo_key == "full":
                oggm_cols = VARS_ORDER[:]  # ['hugonnet_dhdt','consensus_ice_thickness','millan_v']
            else:
                raw = combo_key.split("__")
                oggm_cols = [v for v in VARS_ORDER if v in raw]

            STATIC_COLS = ['aspect', 'slope', 'SVF'] + oggm_cols
            feature_columns = MONTHLY_COLS + STATIC_COLS
            all_columns = feature_columns + cfg.fieldsNotFeatures

            # retain only required + feature columns
            keep = [
                c for c in (set(all_columns) | set(REQUIRED))
                if c in df_grid_monthly.columns
            ]
            df_grid_monthly = df_grid_monthly[keep]

            # required checks
            missing_req = [
                c for c in REQUIRED if c not in df_grid_monthly.columns
            ]
            if missing_req:
                print(
                    f"[{rgi_gl} {year}] missing required cols: {missing_req}, skipping..."
                )
                continue

            # minimal NaN cleanup
            df_grid_monthly_a = df_grid_monthly.dropna(subset=['ID', 'MONTHS'])
            if df_grid_monthly_a.empty:
                print(
                    f"[{rgi_gl} {year}] empty after NaN cleanup, skipping...")
                continue

            # --- build inference dataset with the combo's schema ---
            ds_gl_a = mbm.data_processing.MBSequenceDataset.from_dataframe(
                df_grid_monthly_a,
                MONTHLY_COLS,
                STATIC_COLS,
                months_tail_pad=months_tail_pad,
                months_head_pad=months_head_pad,
                expect_target=False,
                show_progress=False,
            )

            # bring the *matching* training scalers for this combo
            if combo_key not in DS_TRAIN_BY_COMBO:
                print(
                    f"[{rgi_gl} {year}] no training dataset registered for combo '{combo_key}', skipping..."
                )
                continue

            ds_train_for_combo = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
                DS_TRAIN_BY_COMBO[combo_key])

            if combo_key not in SPLIT_IDXS_BY_COMBO:
                print(
                    f"[{rgi_gl} {year}] no split indices registered for combo '{combo_key}', skipping..."
                )
                continue

            train_idx_for_combo, _ = SPLIT_IDXS_BY_COMBO[combo_key]
            ds_train_for_combo.fit_scalers(train_idx_for_combo)

            test_gl_dl_a = mbm.data_processing.MBSequenceDataset.make_test_loader(
                ds_gl_a, ds_train_for_combo, seed=cfg.seed, batch_size=128)

            # --- load + predict with the appropriate model for this combo ---
            try:
                model = get_model_by_combo(combo_key, cfg, device)
            except Exception as e:
                print(
                    f"[{rgi_gl} {year}] model load error for '{combo_key}': {e}"
                )
                continue

            try:
                df_preds_a = model.predict_with_keys(device, test_gl_dl_a,
                                                     ds_gl_a)
            except Exception as e:
                print(
                    f"[{rgi_gl} {year}] prediction error for '{combo_key}': {e}"
                )
                continue

            # --- aggregate to annual glacier mean and write ---
            data_a = df_preds_a[['ID', 'pred']].set_index('ID')
            meta_cols = [
                c for c in ['YEAR', 'POINT_LAT', 'POINT_LON', 'GLWD_ID']
                if c in df_grid_monthly_a.columns
            ]

            grouped_ids_a = (
                df_grid_monthly_a.groupby('ID')[meta_cols].first().merge(
                    data_a, left_index=True, right_index=True, how='left'))
            months_per_id_a = df_grid_monthly_a.groupby(
                'ID')['MONTHS'].unique()
            grouped_ids_a = grouped_ids_a.merge(months_per_id_a,
                                                left_index=True,
                                                right_index=True)

            grouped_ids_a.reset_index(inplace=True)
            grouped_ids_a.sort_values(by='ID', inplace=True)

            pred_y_annual = grouped_ids_a.copy()
            pred_y_annual['PERIOD'] = 'annual'
            pred_y_annual = pred_y_annual.drop(columns=['YEAR'],
                                               errors='ignore')

            mean_MB = pred_y_annual['pred'].mean()

            with open(output_file, 'a') as f:
                f.write(f"{index_counter},{rgi_gl},{year},{mean_MB:.4f}\n")

            index_counter += 1

In [ ]:
# import os, re, traceback, sys
# from concurrent.futures import ProcessPoolExecutor, as_completed
# from datetime import datetime
# import pandas as pd
# from tqdm import tqdm

# # ----------------- config -----------------
# YEAR_RE = re.compile(r"_grid_(\d{4})\.parquet$")
# REQUIRED = ['GLACIER', 'YEAR', 'ID', 'PERIOD', 'MONTHS']

# # Avoid BLAS/OpenMP oversubscription inside each worker
# os.environ.setdefault("OMP_NUM_THREADS", "1")
# os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
# os.environ.setdefault("MKL_NUM_THREADS", "1")
# os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# MAX_WORKERS = min(os.cpu_count() or 4, 12)
# current_date = datetime.now().strftime("%Y%m%d_%H%M")

# # ----------------- helpers -----------------
# def list_years(glacier_path: str, rgi_gl: str):
#     years = []
#     for fname in os.listdir(glacier_path):
#         if fname.endswith(".parquet") and rgi_gl in fname:
#             m = YEAR_RE.search(fname)
#             if m:
#                 years.append(int(m.group(1)))
#     return sorted(set(years))

# def process_one_glacier(rgi_gl: str):
#     """
#     Returns dict:
#       {
#         'rgi': rgi_gl,
#         'rows': [(rgi_gl, year, mean_MB), ...],
#         'log':  ["message", ...],     # warnings/information
#         'err':  "traceback or None"
#       }
#     """
#     out = {'rgi': rgi_gl, 'rows': [], 'log': [], 'err': None}
#     try:
#         seed_all(cfg.seed)

#         glacier_path = os.path.join(path_rgi_alps, rgi_gl)
#         if not os.path.isdir(glacier_path):
#             out['log'].append(f"[{rgi_gl}] folder not found: {glacier_path}")
#             return out

#         years = list_years(glacier_path, rgi_gl)
#         if not years:
#             out['log'].append(f"[{rgi_gl}] no parquet years in {glacier_path}")
#             return out

#         _MODEL_CACHE.clear()
#         local_model_cache = {}

#         for year in years:
#             file_path = os.path.join(glacier_path, f"{rgi_gl}_grid_{year}.parquet")
#             try:
#                 df = pd.read_parquet(file_path)
#             except Exception as e:
#                 out['log'].append(f"[{rgi_gl} {year}] read error: {e}")
#                 continue

#             # normalize SVF column name
#             if 'svf' in df.columns and 'SVF' not in df.columns:
#                 df.rename(columns={'svf': 'SVF'}, inplace=True)

#             # choose combo
#             try:
#                 combo_key = detect_available_combo_key(df)
#             except Exception as e:
#                 out['log'].append(f"[{rgi_gl} {year}] detect combo failed: {e}")
#                 continue

#             if combo_key == "simple":
#                 oggm_cols = []
#             elif combo_key == "full":
#                 oggm_cols = VARS_ORDER[:]   # defined in your config
#             else:
#                 raw = combo_key.split("__")
#                 oggm_cols = [v for v in VARS_ORDER if v in raw]

#             STATIC_COLS = ['aspect', 'slope', 'SVF'] + oggm_cols
#             feature_columns = MONTHLY_COLS + STATIC_COLS
#             all_columns = feature_columns + cfg.fieldsNotFeatures

#             keep = [c for c in (set(all_columns) | set(REQUIRED)) if c in df.columns]
#             df = df[keep]

#             missing_req = [c for c in REQUIRED if c not in df.columns]
#             if missing_req:
#                 out['log'].append(f"[{rgi_gl} {year}] missing required: {missing_req}")
#                 continue

#             df = df.dropna(subset=['ID', 'MONTHS'])
#             if df.empty:
#                 out['log'].append(f"[{rgi_gl} {year}] empty after NaN cleanup")
#                 continue

#             # dataset
#             try:
#                 ds = mbm.data_processing.MBSequenceDataset.from_dataframe(
#                     df,
#                     MONTHLY_COLS,
#                     STATIC_COLS,
#                     months_tail_pad=months_tail_pad,
#                     months_head_pad=months_head_pad,
#                     expect_target=False,
#                     show_progress=False,
#                 )
#             except Exception as e:
#                 out['log'].append(f"[{rgi_gl} {year}] build dataset failed: {e}")
#                 continue

#             # scalers
#             if combo_key not in DS_TRAIN_BY_COMBO or combo_key not in SPLIT_IDXS_BY_COMBO:
#                 out['log'].append(f"[{rgi_gl} {year}] missing train/scaler for combo {combo_key}")
#                 continue

#             ds_train = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
#                 DS_TRAIN_BY_COMBO[combo_key]
#             )
#             train_idx, _ = SPLIT_IDXS_BY_COMBO[combo_key]
#             ds_train.fit_scalers(train_idx)

#             test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
#                 ds, ds_train, seed=cfg.seed, batch_size=128
#             )

#             # model
#             try:
#                 if combo_key not in local_model_cache:
#                     local_model_cache[combo_key] = get_model_by_combo(combo_key, cfg, device)
#                 model = local_model_cache[combo_key]
#             except Exception as e:
#                 out['log'].append(f"[{rgi_gl} {year}] model load error: {e}")
#                 continue

#             # predict
#             try:
#                 df_preds = model.predict_with_keys(device, test_dl, ds)
#             except Exception as e:
#                 out['log'].append(f"[{rgi_gl} {year}] predict error: {e}")
#                 continue

#             mean_MB = float(df_preds['pred'].mean())
#             out['rows'].append((rgi_gl, year, mean_MB))

#         return out

#     except Exception as e:
#         out['err'] = f"{e}\n{traceback.format_exc()}"
#         return out

# # ----------------- driver with progress + incremental write -----------------
# output_file = os.path.join("logs", f"glacier_mean_MB_{current_date}.csv")
# os.makedirs(os.path.dirname(output_file), exist_ok=True)

# # list glacier folders to run
# if not os.path.isdir(path_rgi_alps):
#     print(f"❌ path_rgi_alps not found: {path_rgi_alps}")
# else:
#     rgi_id_list = [d for d in os.listdir(path_rgi_alps)
#                    if os.path.isdir(os.path.join(path_rgi_alps, d))]
#     print(f"Will process {len(rgi_id_list)} glacier folders under {path_rgi_alps}")
#     if rgi_id_list[:5]:
#         print("First few:", rgi_id_list[:5])

# # write header
# with open(output_file, 'w') as f:
#     f.write("Index,RGIId,Year,Mean_MB\n")
# index_counter = 0

# # run
# with ProcessPoolExecutor(max_workers=MAX_WORKERS) as ex:
#     futs = {ex.submit(process_one_glacier, rgi): rgi for rgi in rgi_id_list}
#     with tqdm(total=len(futs), desc="Glaciers", ncols=100) as pbar:
#         for fut in as_completed(futs):
#             rgi = futs[fut]
#             try:
#                 result = fut.result()
#             except Exception as e:
#                 print(f"[{rgi}] worker crashed: {e}")
#                 pbar.update(1)
#                 continue

#             # log diagnostics
#             for msg in result.get('log', []):
#                 print(msg)
#             if result.get('err'):
#                 print(f"[{rgi}] ERROR:\n{result['err']}")

#             rows = result.get('rows', [])
#             if rows:
#                 with open(output_file, 'a') as f:
#                     for rgi_gl, year, mean_MB in rows:
#                         f.write(f"{index_counter},{rgi_gl},{year},{mean_MB:.4f}\n")
#                         index_counter += 1
#                 # ensure it hits disk even if kernel dies
#                 try:
#                     os.fsync(open(output_file, 'a').fileno())
#                 except Exception:
#                     pass

#             pbar.set_postfix_str(f"{rgi}: {len(rows)} rows")
#             pbar.update(1)

# print(f"✅ Done. Wrote {index_counter} lines to {output_file}")

## Results:

In [ ]:
#path_rgi = cfg.dataPath+'GLAMOS/RGI/RGI2000-v7.0-G-11_central_europe/RGI2000-v7.0-G-11_central_europe.shp'
path_rgi = cfg.dataPath + 'GLAMOS/RGI/nsidc0770_11.rgi60.CentralEurope/11_rgi60_CentralEurope.shp'

# load RGI shapefile
gdf = gpd.read_file(path_rgi)

# check CRS
print(gdf.crs)

# reproject to a local equal-area projection (example: EPSG:3035 for Europe)
gdf_proj = gdf.to_crs(3035)
gdf_proj.set_index('RGIId', inplace=True, drop=True)
gdf_proj["area_m2"] = gdf_proj.geometry.area
gdf_proj["area_km2"] = gdf_proj["area_m2"] / 1e6

In [ ]:
# open output file
output_df = pd.read_csv("logs/glacier_mean_MB_2025-10-08.csv").drop(['Index'], axis=1)

output_df['area_gl'] = output_df['RGIId'].map(
    lambda x: gdf_proj.loc[x, 'area_km2'])

# yearly_mean_mb_CA = output_df.groupby('Year',
#                                       as_index=False).agg({'Mean_MB': 'mean'})
# yearly_cum_mb_CA = output_df.groupby('Year',
#                                      as_index=False).agg({'Mean_MB': 'sum'})
# yearly_cum_mb_CA['Cum_MB'] = yearly_cum_mb_CA['Mean_MB'].cumsum()
# yearly_cum_mb_CA['Mean_MB'] = yearly_mean_mb_CA['Mean_MB']
# # yearly_cum_mb_CA['Mean_MB'] = yearly_cum_mb_CA['Mean_MB'] / total_area
# yearly_cum_mb_CA.head()

df = output_df.copy()

# annual change per glacier in Gt
df["annual_change_gt"] = (df["Mean_MB"] * df["area_gl"]) / 1e9

# total annual change in Gt (sum across glaciers)
annual_gt = df.groupby("Year")["annual_change_gt"].sum().reset_index(
    name="Annual_MB_Gt")

# cumulative MB in Gt
annual_gt["Cumulative_MB_Gt"] = annual_gt["Annual_MB_Gt"].cumsum()

# compute weighted mean MB per year
yearly_weighted = (output_df.groupby("Year").apply(lambda g: (g["Mean_MB"] * g[
    "area_gl"]).sum() / g["area_gl"].sum()).reset_index(name="Weighted_MB"))

print(yearly_weighted.head())

In [ ]:
output_df

In [ ]:
glambie_df = pd.read_csv('glambie_values.csv')
date_columns = [
    'central_europe_dates', 'central_europe_start_dates',
    'central_europe_end_dates'
]

glambie_df[date_columns] = glambie_df[date_columns].apply(
    lambda x: x.round() - 1)
glambie_df.head()

In [ ]:
# --- plotting ---
fig, axs = plt.subplots(1, 2, figsize=(15, 6), sharey=True)

# --------------------
# Left: LSTM results
# --------------------
ax1 = axs[0]
years = yearly_weighted['Year']

# barplot: annual weighted MB (m w.e.)
ax1.bar(years,
        yearly_weighted['Weighted_MB'],
        color="skyblue",
        label="Area-weighted annual MB")
ax1.set_ylabel("Annual MB (m w.e.)", color="skyblue")

# lineplot: cumulative MB in Gt (secondary axis)
ax2 = ax1.twinx()
ax2.plot(annual_gt['Year'],
         annual_gt['Cumulative_MB_Gt'],
         color="red",
         marker="o",
         label="Cumulative MB")
ax2.set_ylabel("Cumulative MB (Gt)", color="red")

ax1.set_title("Central Alps annual MB (LSTM)")
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

# --------------------
# Right: GLAMBIE results
# --------------------
ax3 = axs[1]

# annual MB (bars)
ax3.bar(glambie_df['central_europe_end_dates'],
        glambie_df['central_europe_annual_change_mwe'],
        color="lightgreen",
        label="Annual MB (GLAMBIE)")
ax3.set_ylabel("Annual MB (m w.e.)", color="lightgreen")

# cumulative MB (line, secondary axis)
ax4 = ax3.twinx()
ax4.plot(glambie_df['central_europe_dates'],
         glambie_df['central_europe_cumulative_change_gt'],
         color="darkgreen",
         marker="s",
         label="Cumulative MB (GLAMBIE)")
ax4.set_ylabel("Cumulative MB (Gt)", color="darkgreen")

ax3.set_title("Central Europe MB (GLAMBIE)")
ax3.legend(loc="upper left")
ax4.legend(loc="upper right")

# --------------------
# Formatting
# --------------------
for ax in axs:
    ax.tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# make sure both datasets use the same x-axis type
years_lstm = yearly_weighted['Year']

years_glambie = glambie_df['central_europe_end_dates']

fig, ax = plt.subplots(figsize=(12, 6))

# bar width
width = 0.4

# LSTM bars (slightly shifted left)
ax.bar(years_lstm - 0.2,
       yearly_weighted['Weighted_MB'],
       width=width,
       color="skyblue",
       label="LSTM Annual MB")

# GLAMBIE bars (slightly shifted right)
ax.bar(years_glambie + 0.2,
       glambie_df['central_europe_annual_change_mwe'],
       width=width,
       color="lightgreen",
       label="GLAMBIE Annual MB")

# formatting
ax.set_ylabel("Annual MB (m w.e.)")
ax.set_title("Annual Mass Balance: LSTM vs GLAMBIE")
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()